In [1]:
from keras.applications.mobilenet_v2 import preprocess_input
from keras.utils import img_to_array
from keras.models import load_model
import numpy as np
import imutils
import time
import cv2 as cv
import os

2022-10-08 09:55:09.318414: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-08 09:55:09.318476: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
os.chdir('../../data/Face-Mask-Detection')

In [3]:
def detect_mask(frame, faceNet, maskNet):
    h, w = frame.shape[:2]
    blob = cv.dnn.blobFromImage(frame, 1.0, (224, 224), (104.0, 177.0, 123.0))

    faceNet.setInput(blob)
    detections = faceNet.forward()

    faces = []
    locs = []
    preds = []

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            startx, starty, endx, endy = box.astype('int')

            startx, starty = max(0, startx), max(0, starty)
            endx, endy = min(w-1, endx), min(h-1, endy)

            face = frame[starty:endy, startx:endx]
            face = cv.cvtColor(face, cv.COLOR_BGR2RGB)
            face = cv.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            faces.append(face)
            locs.append((startx, starty, endx, endy))
    if len(faces)>0:
        faces = np.array(faces, dtype='float32')
        preds = maskNet.predict(faces, batch_size=32, verbose=0)
    return locs, preds

In [4]:
prototxtPath = 'face_detector/deploy.prototxt'
weightsPath = 'face_detector/res10_300x300_ssd_iter_140000.caffemodel'

faceNet = cv.dnn.readNet(prototxtPath, weightsPath)
maskNet = load_model('mask_detector.model')

2022-10-08 09:55:13.621464: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2022-10-08 09:55:13.621541: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: sheded-Latitude-E6520
2022-10-08 09:55:13.621554: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: sheded-Latitude-E6520
2022-10-08 09:55:13.622128: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 390.154.0
2022-10-08 09:55:13.622210: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 390.154.0
2022-10-08 09:55:13.622221: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 390.154.0


In [ ]:
cap = cv.VideoCapture(1)
fourcc = cv.VideoWriter_fourcc(*'MP4V')
writer = cv.VideoWriter('mask_detection.mp4', fourcc, 30, (400, 300))
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        locs, preds = detect_mask(frame, faceNet, maskNet)

        for box, pred in zip(locs, preds):
            startx, starty, endx, endy = box
            mask, withoutMask = pred

            label = 'Mask' if mask > withoutMask else 'No Mask'
            color = (0, 255, 0) if label == 'Mask' else (0, 0, 255)

            label = '{}: {:.2f}%'.format(label, max(mask, withoutMask) * 100)

            cv.putText(frame, label, (startx, starty-10), cv.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv.rectangle(frame, (startx, starty), (endx, endy), color, 2)
        writer.write(frame)
        cv.imshow('frame', frame)
        if cv.waitKey(20) ==27:
            break
    else:
        break
writer.release()
cap.release()
cv.destroyAllWindows()